In [1]:
import numpy as np
import osmnx as ox
import geopandas as gpd
import joblib
from collections import Counter
from copy import deepcopy
import time
import os
from sklearn.metrics import pairwise_distances

from patrol_routes import PatrolRoutes

from pyhotspot_auxiliar import (
    quartic_gaussian_distance, 
    degree_per_meter,
    multigraph_to_graph,
    nearest_edges_index,
    plot_network_events_folium
)

from patrol_routes import (
    round_trip_size, 
    patrol_routes_metrics
)

from ipyleaflet_layers import (
    graph_to_gdfs,
    ipyleaflet_map_geometry, 
    ipyleaflet_clusterize_hot_segments, 
    ipyleaflet_routes_with_statistics
)

# 1. Load and process data

In [2]:
## 1.1. List the cities available

In [3]:
# list cities available
cities_fp = 'cities/'
available_cities = os.listdir(cities_fp)
available_cities

['Chicago_district_1st', 'Chicago_district_12th', 'Chicago_district_18th']

In [4]:
## 1.2. Select the citie

In [5]:
# ----- Select your city ----------
city_name = 'Chicago_district_12th'
# ---------------------------------

In [6]:
prefix_fp = cities_fp + city_name + '/'

# Load events
fp_events =  prefix_fp + 'Events_' + city_name + '.json'
gdf_events = gpd.read_file(fp_events)

# Load network graph
fp_graph = prefix_fp + 'Multidigraph_' + city_name + '.sav'
city_graph = joblib.load(fp_graph)

# Load area perimeter
fp_area = prefix_fp + 'Area_' + city_name + '.json'
city_area = gpd.read_file(fp_area)
city_polygon = city_area.iloc[0].geometry

In [7]:
# Transform network to undirected and remove multiedges
city_graph = ox.get_undirected(city_graph)
city_graph = multigraph_to_graph(city_graph, weight='length', min_weight=True)

# 2. Count the crime events to network segments

In [8]:
edges_city = graph_to_gdfs(city_graph, nodes=False)

# get the proportional degree by meter based on latitude of region
degree_meter = (1/degree_per_meter(gdf_events.Latitude.mean()))

edges_idx, edges_distance = nearest_edges_index(edges_city, gdf_events, method='balltree', 
                                                dist=5*degree_meter, return_distance=True)

In [9]:
# Filter distances shorter than mindist
mindist_filter = 100 #m

dist_filter = edges_distance < (mindist_filter * degree_meter)
num_events_old = len(gdf_events)
edges_idx = edges_idx[dist_filter]
gdf_events = gdf_events[dist_filter]
num_events_new = len(gdf_events)

print("Number of events removed by distance {} m = {}".format(mindist_filter, 
                                                              num_events_old - num_events_new))

Number of events removed by distance 100 m = 1


In [10]:
counter = Counter(edges_idx)
score = np.zeros(edges_city.shape[0])
score[list(counter.keys())] = list(counter.values())

edges_city['score'] = score
gdf_events['nearest_edge'] = edges_idx

# 3. Include Kernel Density Estimation for tiebreaker

In [11]:
bandwidth = 600 * degree_meter # 600m

geo_centroids = np.array([edges_city.centroid.x, edges_city.centroid.y]).T
events = np.array([gdf_events.geometry.x.values, gdf_events.geometry.y.values]).T

distances = pairwise_distances(geo_centroids, events)

kde_score = np.sum(quartic_gaussian_distance(distances, bandwidth), axis=1)

edges_city['kde'] = kde_score

/home/carlos/miniconda3/envs/patrolroutes/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


# 4. Sort edges by score and density estimation

In [12]:
edges_city = edges_city.sort_values(['score', 'kde'], ascending=(False, False))

In [13]:
# ploting
select_edges = edges_city.iloc[:100]

plot_network_events_folium(city_polygon, gdf_events, select_edges, 
                           maximum_cluster=10)

# 5. Patrol Routes Algorithm

## 5.1. Set the parameters

In [14]:
# ----- PARAMATERS --------
minimum_length = 700
maximum_length = 1200
percentage_of_crimes = 0.5
num_routes = 10
direct_path = False
# -------------------------

## 5.2. Select the hotsegments

In [15]:

for index, (u, v, data) in enumerate(city_graph.edges(data=True)):
    data['score'] = score[index]

total_of_crimes = len(gdf_events)

# Number of crimes inside concentration
crimes_to_reach = total_of_crimes * percentage_of_crimes

# Select the highest values of crimes until reach the crimes_to_reach
sum_values = 0
for index_road, value in enumerate(edges_city['score']):
    sum_values += value
    if sum_values >= crimes_to_reach:
        break
        
select_roads = edges_city.iloc[:index_road+1]

hotsegments = [(edge.u, edge.v) for index, edge in 
                select_roads.iterrows()]

## 5.3. Run the algorithm

In [16]:
aux_graph = deepcopy(city_graph)
start_time = time.time()
# -----------------------

pr = PatrolRoutes(aux_graph, debug=False)
pr = pr.create_best_routes(num_routes, hotsegments, minimum_length, 
                           maximum_length, direct_path=direct_path)
routes_list = pr.patrol_routes_list

# -----------------------
execution_time = time.time() - start_time
print("Runtime (seconds): {:.2f}".format(execution_time))

Runtime (seconds): 340.31


## 5.4. Calculate the metrics

In [17]:
metrics = patrol_routes_metrics(routes_list, city_graph, hotsegments, weight_length='length',
                                weight_score='score', percentage=True)

routes_length_vec = [round_trip_size(route) for route in routes_list]
print("Number of routes", len(routes_list))
print("Minimum Routes Length:", min(routes_length_vec))
print("Maximum Routes Length:", max(routes_length_vec))

metrics

Number of routes 10
Minimum Routes Length: 841.5679999999999
Maximum Routes Length: 1184.334


,metrics
Mean of patrol routes length (std),1078 (91.95)
Patrol routes score (%),392 (19.47%)
Hotsegments covered score (%),376 (37.34%)
Number hotsegments covered (%),66 (23.40%)
Patrol length as hotsegment (%),5902 (54.73%)
Patrol cycle length (%),9992 (97.49%)
Network PAI,14.35


# 6. Show the patrol routes

In [18]:
ipymap = ipyleaflet_map_geometry(city_polygon)

layers = ipyleaflet_clusterize_hot_segments(select_roads, gdf_events)
for layer in layers.values():
    ipymap.add_layer(layer)
    
layers = ipyleaflet_routes_with_statistics(routes_list, gdf_events, city_graph, round_trip_length=True)
ipymap.add_layer(layers['routes'])
ipymap.add_layer(layers['statistics'])

/home/carlos/Desktop/doutorado/PatrolRoutes/Experimental results/ipyleaflet_layers.py:726: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  marker = ipyleaflet.Marker(location = [hotspot_edges.centroid.y.mean(),
/home/carlos/Desktop/doutorado/PatrolRoutes/Experimental results/ipyleaflet_layers.py:727: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hotspot_edges.centroid.x.mean()],popup=message, popup_max_width=130,


In [19]:
ipymap

Map(center=[41.87702776413797, -87.66919963516776], controls=(ZoomControl(options=['position', 'zoom_in_text',…